In [8]:
import pyreadr
import numpy as np
import pandas as pd
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier

# <u>Preprocess Data </u>

### Import Transcript Stability Data

In [9]:
transcript_stability = pyreadr.read_r('transcript_stability_MAD_count_norm_0.8.RData')
transcript_stability = transcript_stability["transcript_stability"] # 84107 rows
transcript_stability = pd.DataFrame(transcript_stability)

### Remove transcript version from id's (*.number)

In [10]:
transcript_stability["transcript_id"] = transcript_stability["transcript_id"].str.split(".").str[0]

### Remove Housekeeping transcripts

In [11]:
h = pd.read_csv("Housekeeping_TranscriptsHuman.csv", delimiter=";")
house_genes = h["Ensembl"].values
transcript_stability = transcript_stability[~transcript_stability["transcript_id"].isin(house_genes)]

### Extract median expression & spearman stability only 

In [12]:
transcript_stability = transcript_stability.loc[:,['combined_median','stability']]

# <u>Build Random Forest classifier </u>

## 1. Balanced Random Forest Classifier with k-fold cross validation

- Will downsample majority class (unstable transcripts)
- For each tree in RF, down-sample the majority class to the same size as the minority class
- Given enough trees, all training data are used, so no loss of information
- Computationally efficient, since each tree only sees a small sample


In [13]:
x1 = transcript_stability["combined_median"].values.reshape(-1, 1) # convert into 2D array
y1 = transcript_stability["stability"].values.ravel() # convert into 1D array
brf = BalancedRandomForestClassifier(n_estimators=500, random_state=42) #define classifier with 500 trees
k_fold = StratifiedKFold(n_splits=5, random_state=42, shuffle=True) #stratified better for unbalanced datasets; 5-fold CV
brf_scores = cross_val_score(brf, x1, y1, cv=k_fold, n_jobs=2, pre_dispatch=4)

In [14]:
print("Accuracy: %.2f%%" % (brf_scores.mean()*100.0))

Accuracy: 68.02%


### Obtain confusion matrix for predicted values

In [15]:
brf_pred = cross_val_predict(brf, x1, y1, cv=5, n_jobs=3, pre_dispatch=6) #5-fold CV
accuracy = accuracy_score(y1, brf_pred)
labels = np.unique(y1)
conf_mat = confusion_matrix(y1, brf_pred, labels=labels)

In [16]:
pd.DataFrame(conf_mat, index=labels, columns=labels)

,NO,YES
NO,54648,25896
YES,410,1009


## 2. Weighted Random Forest Classifier with k-fold cross validation

- Place a heavier penalty on misclassifying the minority class (stable transcripts), using class_weight

In [19]:
wrf = RandomForestClassifier(n_estimators=500, random_state=42, class_weight='balanced')
cv_wrf = StratifiedKFold(n_splits=5)
wrf_pred = cross_val_predict(wrf, x1, y1, cv=10, n_jobs=3, pre_dispatch=6)

### Obtain confusion matrix for predicted values

In [20]:
accuracy = accuracy_score(y1, wrf_pred)
labels = np.unique(y1)
conf_wrf = confusion_matrix(y1, wrf_pred, labels=labels) #extract confusion matrix

In [23]:
print(accuracy * 100)

96.36543318326562


In [21]:
pd.DataFrame(conf_wrf, index=labels, columns=labels)

,NO,YES
NO,78909,1635
YES,1344,75
